In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-10-30"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
10236,2024-10-30,Brasil Nbb,19:30,Unifacisa,Botafogo,1.20,3.84,153.5,1.83,1.83,-10.5,1.92,1.75,hr1AyBWs,0.833333,0.260417,0.546448,0.546448,0.093750,0.0,0.2,NaN,NaN,120.470,34.167913,0.283622,203.870,90.449008,0.443660,96.828,261.890,72.45,246.40,0.0,0.0,1.0,0.0,0.740779,0.000000,0.065509,-2.66,-0.532,-0.375940,0.737360,0.7,-0.037360,6.71,1.342,2.116244,0.295416,0.5,0.204584
10237,2024-10-30,Brasil Nbb,20:00,São Paulo,Paulistano,1.85,1.81,151.5,1.86,1.80,-1.5,2.03,1.67,084IZl1f,0.540541,0.552486,0.537634,0.555556,0.093027,0.6,0.2,NaN,NaN,150.780,42.840836,0.284128,124.952,37.627646,0.301137,172.264,142.740,107.64,89.78,0.0,0.0,0.0,0.0,0.015456,0.023184,0.137599,1.13,0.226,3.761062,0.580588,0.4,-0.180588,3.01,0.602,1.345515,0.633514,0.8,0.166486
10238,2024-10-30,Eua Nba,20:00,Charlotte Hornets,Toronto Raptors,1.55,2.56,225.5,1.81,1.99,-4.5,1.93,1.93,l4TuoG2S,0.645161,0.390625,0.552486,0.502513,0.035786,0.2,0.6,NaN,NaN,329.298,30.752659,0.093389,387.860,179.396624,0.462529,191.370,304.014,279.84,667.61,0.0,1.0,0.0,0.0,0.347532,0.066989,0.000000,-1.53,-0.306,-1.797386,0.393626,0.5,0.106374,-3.23,-0.646,-2.414861,0.476639,0.2,-0.276639
10239,2024-10-30,Eua Nba,20:00,Cleveland Cavaliers,Los Angeles Lakers,1.44,2.51,227.5,1.86,1.81,-6.5,2.01,1.68,KlLILeuf,0.694444,0.398406,0.537634,0.552486,0.092851,0.8,0.2,NaN,NaN,189.346,43.542135,0.229961,223.734,11.231649,0.050201,210.794,278.646,134.47,234.15,0.0,0.0,0.0,0.0,0.383091,0.019267,0.126474,-3.81,-0.762,-0.577428,0.568161,0.4,-0.168161,-0.69,-0.138,-10.942029,0.456132,0.5,0.043868
10240,2024-10-30,Eua Nba,20:00,Indiana Pacers,Boston Celtics,3.19,1.28,231.5,1.83,1.83,5.5,2.01,1.68,GGERJZ96,0.313480,0.781250,0.546448,0.546448,0.094730,0.8,0.4,NaN,NaN,226.980,70.693459,0.311452,154.964,24.701674,0.159403,206.204,149.712,186.96,132.68,0.0,0.0,0.0,0.0,0.604284,0.000000,0.126474,-3.59,-0.718,-3.050139,0.660922,0.6,-0.060922,-1.12,-0.224,-1.250000,0.722429,0.8,0.077571
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10350,2024-10-30,Lituânia King Mindaugas Cup,14:00,Neptunas,Juventus,1.72,1.95,169.5,1.83,1.83,-2.5,1.95,1.72,lY6hNiIP,0.581395,0.512821,0.546448,0.546448,0.094216,0.0,0.0,NaN,NaN,128.472,31.303772,0.243662,130.776,40.353662,0.308571,0.000,0.000,108.36,201.67,0.0,0.0,0.0,0.0,0.088629,0.000000,0.088629,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
10351,2024-10-30,República Tcheca Zbl Feminina,15:00,Trutnov F,LOH 2028 Chomutov F,1.01,16.00,145.5,1.83,1.83,-27.5,1.92,1.78,MBpOrRBM,0.990099,0.062500,0.546448,0.546448,0.052599,0.4,0.0,NaN,NaN,210.030,223.060089,1.062039,NaN,NaN,NaN,238.140,0.000,109.80,140.07,0.0,0.0,0.0,0.0,1.246271,0.000000,0.053511,-0.94,-0.188,-0.053191,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
10352,2024-10-30,Rússia Premier League Feminina,11:00,Nadezhda F,UMMC Ekaterinburg F,9.80,1.03,138.5,1.86,1.84,15.5,2.03,1.60,EXcxVyML,0.102041,0.970874,0.537634,0.543478,0.072915,0.2,0.6,NaN,NaN,135.758,74.396011,0.548005,532.208,960.590004,1.804915,119.192,843.966,133.65,84.84,0.0,0.0,0.0,0.0,1.145213,0.007644,0.167524,1.98,0.396,22.222222,0.000000,0.0,0.000000,44.12,8.824,0.003400,0.000000,0.0,0.000000
10353,2024-10-30,Rússia Premier League Feminina,12:00,Nika Siktivkar F,Novosibirsk F,1.20,4

## Apostas do dia

### Over v0 (LDA)

In [9]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,20:00,Eua Nba,Charlotte Hornets,Toronto Raptors,225.5,1.81,0.5823,Over
1,20:00,Eua Nba,Washington Wizards,Atlanta Hawks,233.5,1.84,0.8652,Over
2,21:00,Eua Nba,Chicago Bulls,Orlando Magic,226.5,1.84,0.9824,Over
3,21:00,Eua Nba,Memphis Grizzlies,Brooklyn Nets,224.5,1.83,0.9139,Over
4,23:00,Eua Nba,Golden State Warriors,New Orleans Pelicans,218.5,1.88,0.8787,Over
